In [1]:
import coiled
import dask.distributed
import dask.dataframe as dd

## Cluster setup

In [2]:
cluster = coiled.Cluster(configuration="coiled/default", n_workers=5)

Output()

Found software environment build


In [3]:
client = dask.distributed.Client(cluster)

## Resonably sized CSV files

In [4]:
d_7d_2000 = dd.read_csv("s3://coiled-datasets/timeseries/7d/csv/2000/*.csv")
d_7d_2001 = dd.read_csv("s3://coiled-datasets/timeseries/7d/csv/2001/*.csv")

In [5]:
ddf_7d = dd.concat([d_7d_2000, d_7d_2001])

In [21]:
len(ddf_7d)

62899200

In [6]:
ddf_7d.npartitions

104

In [7]:
%%time

ddf_7d["name"].nunique().compute()

CPU times: user 202 ms, sys: 71 ms, total: 273 ms
Wall time: 24.9 s


26

## Tiny CSV files

In [8]:
d_1h_2000 = dd.read_csv("s3://coiled-datasets/timeseries/1h/csv/2000/*.csv")
d_1h_2001 = dd.read_csv("s3://coiled-datasets/timeseries/1h/csv/2001/*.csv")

In [9]:
ddf_1h = dd.concat([d_1h_2000, d_1h_2001])

In [10]:
ddf_1h.npartitions

17496

In [11]:
%%time

ddf_1h["name"].nunique().compute()

CPU times: user 6.62 s, sys: 648 ms, total: 7.26 s
Wall time: 7min 34s


26

## Repartition tiny CSVs then count

In [16]:
%%time

repartitioned = ddf_1h.repartition(100)

CPU times: user 7.99 ms, sys: 989 µs, total: 8.98 ms
Wall time: 8.93 ms


In [17]:
%%time

repartitioned["name"].nunique().compute()

CPU times: user 5.88 s, sys: 666 ms, total: 6.54 s
Wall time: 7min 3s


26

In [19]:
%%time

repartitioned.to_parquet("s3://coiled-datasets/tmp/repartitioned")

CPU times: user 5.72 s, sys: 672 ms, total: 6.4 s
Wall time: 7min 27s


In [22]:
len(dd.read_parquet("s3://coiled-datasets/tmp/repartitioned"))

62985600

In [20]:
%%time

ddf = dd.read_parquet("s3://coiled-datasets/tmp/repartitioned")
ddf["name"].nunique().compute()

CPU times: user 141 ms, sys: 79.5 ms, total: 221 ms
Wall time: 8.76 s


26

## Ressonably sized Parquet files

In [12]:
p_7d_2000 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2000")
p_7d_2001 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2001")

In [13]:
p_ddf_7d = dd.concat([p_7d_2000, p_7d_2001])

In [14]:
p_ddf_7d.npartitions

104

In [15]:
%%time

p_ddf_7d["name"].nunique().compute()

CPU times: user 85.6 ms, sys: 19 ms, total: 105 ms
Wall time: 13.8 s


26

## Show Dataset

In [18]:
p_7d_2000.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930
